# Day 12

## Part 1

In [79]:
import collections
# import time

with open("paths.txt") as f:
    lines = [line.strip() for line in f.readlines()]
    pairs = [(i.split("-")[0], i.split("-")[1]) for i in lines] 
    
print(pairs)

big = set()
small = set()

paths = collections.defaultdict(list)

for pair in pairs:
    for cave in pair:
        if cave not in ("start", "end"):
            if cave == cave.upper():
                big.add(cave)
            else:
                small.add(cave)
    paths[pair[0]].append(pair[1])
    paths[pair[1]].append(pair[0])
    
del paths["end"]  # don't need to know what's linked after the end
    
print(big)
print(small)
print(paths)

[('zi', 'end'), ('XR', 'start'), ('zk', 'zi'), ('TS', 'zk'), ('zw', 'vl'), ('zk', 'zw'), ('end', 'po'), ('ws', 'zw'), ('TS', 'ws'), ('po', 'TS'), ('po', 'YH'), ('po', 'xk'), ('zi', 'ws'), ('zk', 'end'), ('zi', 'XR'), ('XR', 'zk'), ('vl', 'TS'), ('start', 'zw'), ('vl', 'start'), ('XR', 'zw'), ('XR', 'vl'), ('XR', 'ws')]
{'XR', 'YH', 'TS'}
{'zw', 'vl', 'xk', 'ws', 'zi', 'zk', 'po'}
defaultdict(<class 'list'>, {'zi': ['end', 'zk', 'ws', 'XR'], 'XR': ['start', 'zi', 'zk', 'zw', 'vl', 'ws'], 'start': ['XR', 'zw', 'vl'], 'zk': ['zi', 'TS', 'zw', 'end', 'XR'], 'TS': ['zk', 'ws', 'po', 'vl'], 'zw': ['vl', 'zk', 'ws', 'start', 'XR'], 'vl': ['zw', 'TS', 'start', 'XR'], 'po': ['end', 'TS', 'YH', 'xk'], 'ws': ['zw', 'TS', 'zi', 'XR'], 'YH': ['po'], 'xk': ['po']})


In [ ]:
# if a small cave only links to a single small cave, then it's useless
# since it's just an end point and you can't get to it
# so erase it from existence
# THIS MESSES UP PART 2 SO I'M COMMENTING IT OUT :(

# to_remove = []
# for cave in small:
#     if len(paths[cave]) == 1 and (paths[cave][0] == paths[cave][0].lower()):
#         print(cave, paths[cave], len(paths[cave]), paths[cave][0])
#         paths[paths[cave][0]].remove(cave)
#         del paths[cave]
#         to_remove.append(cave)
        
# for cave in to_remove:
#     small.remove(cave)
        
# print(big)
# print(small)
# print(paths)

In [80]:
# need to find way from arbitrary cave to end

def travel_cave(cave_name, travelled, full_paths):
    """
    Recursively search through caves for the end.
    Need to know current cave searching, the path so far, and list to add completed paths to.
    """
    
    # print(cave_name, paths[cave_name])
    # print(travelled, full_paths)
    # print()
    
    # time.sleep(0.25)
    travelled = travelled[:]
    travelled.append(cave_name)  # add current cave to list of those travelled
    cave_list = paths[cave_name]  # get list of caves linked from current cave_name
    
    for cave in cave_list:
        # go through each cave linking from this one
        if cave == "end":
            # end found
            path = travelled[:]
            path.append("end")
            # print("complete path", path)
            full_paths.append(path)  # add completed path to list of completed paths

        elif (cave not in travelled) or (cave == cave.upper()):
            # if cave hasn't been visited so fat, or it's a big cave, can still explore
            other_paths = travel_cave(cave, travelled, full_paths)
            # full_paths.append(other_paths)
            
    return full_paths
    
full_paths = []
travelled = []
full_paths = travel_cave("start", travelled, full_paths)
    
print("completed?? maybe??")
# print(full_paths)
print("Number of paths found:", len(full_paths))

completed?? maybe??
Number of paths found: 3761


## Part 2

In [94]:
# need to find way from arbitrary cave to end

def travel_cave_2(cave_name, travelled, full_paths, visit_twice):
    """
    Recursively search through caves for the end.
    Need to know current cave searching, the path so far, and list to add completed paths to.
    Now, I can visit a single small cave twice in each path.
    Pass along the current "visit twice" cave?.
    """
    
    # print(cave_name, paths[cave_name], travelled)
    # print(travelled, full_paths)
    # print("visit twice:", visit_twice)
    
    # time.sleep(0.25)
    travelled = travelled[:]
    travelled.append(cave_name)  # add current cave to list of those travelled
    cave_list = paths[cave_name]  # get list of caves linked from current cave_name
    
    for cave in cave_list:
        # go through each cave linking from this one
        if cave == "end":
            # end found
            full_paths.append(travelled[:] + ["end"])
            # path = travelled[:]
            # path.append("end")
            # # print("complete path", path)
            # full_paths.append(path)  # add completed path to list of completed paths

        else:
            # next cave isn't the end
            if cave in big:
                # it's a big cave, can still explore
                other_paths = travel_cave_2(cave, travelled, full_paths, visit_twice)

            else: 
                # it's a small cave - can only visit a single small cave twice
                # check how many times it's been visited. if only visited once
                # and no cave has been set as the visit-twice cave, go to it
                # and set it as the visit-twice cave?
                
                # skip "start" cave lol
                if cave == "start":
                    continue
                
                # first, count how many times this cave has been visited
                if travelled.count(cave) == 0:
                    # if not visited yet, act as normal
                    other_paths = travel_cave_2(cave, travelled, full_paths, visit_twice)
                elif travelled.count(cave) == 1:
                    # here's the tricky part
                    # the cave it's looking at has already been visited once
                    # check: has a cave been visited twice already?
                    if len(visit_twice) != 0:
                        # another cave has already been visited twice, so don't do anything
                        pass
                    else:
                        # this is the second time seeing this cave, but no other cave has been visited twice
                        # note that this is the visit-twice cave, then keep going                        
                        other_paths = travel_cave_2(cave, travelled, full_paths, [cave])
                elif travelled.count(cave) == 2:
                    # have visited the next cave twice already
                    # don't do anything
                    pass
                
            
    return full_paths
    
full_paths = []
travelled = []
visit_twice = []

import time

start_time = time.time()
full_paths = travel_cave_2("start", travelled, full_paths, visit_twice)
print(time.time() - start_time)
    
print("completed?? maybe??")
print("Number of paths found:", len(full_paths))  # should get 36

# for path in full_paths:
#     print(path)

3.855966329574585
completed?? maybe??
Number of paths found: 99138


The cell below was used to check which solutions I wasn't getting compared to the first example, for help debugging.

In [65]:
with open("ex1_p2_paths.txt") as f:
    lines = [line.strip().split(",") for line in f.readlines()]
    
# I'm not getting some paths D: why not??
for path in full_paths:
    if path not in lines:
        print("GETTING SOLUTION THAT ISN'T VALID")
        print(path)

for line in lines:
    if line not in full_paths:
        print(line)

['start', 'A', 'b', 'A', 'c', 'A', 'b', 'A', 'end']
['start', 'A', 'b', 'A', 'c', 'A', 'b', 'end']
['start', 'A', 'c', 'A', 'b', 'A', 'b', 'A', 'end']
['start', 'A', 'c', 'A', 'b', 'A', 'b', 'end']
['start', 'A', 'c', 'A', 'b', 'A', 'c', 'A', 'end']
['start', 'A', 'c', 'A', 'b', 'd', 'b', 'A', 'end']
['start', 'A', 'c', 'A', 'b', 'd', 'b', 'end']
['start', 'b', 'A', 'c', 'A', 'b', 'A', 'end']
['start', 'b', 'A', 'c', 'A', 'b', 'end']
